In [1]:
import sys, os
sys.path.append(os.path.abspath(".."))

In [2]:
import cgael
from cgael.models.SimpleColor import *

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layer

import pygad
import pygad.kerasga

import numpy as np
import pandas as pd

In [3]:
ts = cgael.LanguageTokenSet("CHAT", '-')
gen = SimpleColorGenerator([Swatch.WHITE, Swatch.BLACK], blur=0, batch_lock=True)
model = SimpleColorModel(token_set=ts, word_count=1, word_length=3)

In [4]:
model.speaker.summary()

Model: "speaker"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 3)]            0         
                                                                 
 flatten (Flatten)           (None, 3)                 0         
                                                                 
 dense (Dense)               (None, 15)                60        
                                                                 
 reshape (Reshape)           (None, 1, 3, 5)           0         
                                                                 
Total params: 60 (240.00 Byte)
Trainable params: 60 (240.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
model.listener.summary()

Model: "listener"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1, 3)]            0         
                                                                 
 embedding (Embedding)       (None, 1, 3, 4)           20        
                                                                 
 flatten_1 (Flatten)         (None, 12)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 39        
                                                                 
 reshape_1 (Reshape)         (None, 1, 3)              0         
                                                                 
Total params: 59 (236.00 Byte)
Trainable params: 59 (236.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
model.model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1, 3)]            0         
                                                                 
 speaker (Functional)        (None, 1, 3, 5)           60        
                                                                 
 argmax_layer (ArgmaxLayer)  (None, 1, 3)              0         
                                                                 
 language_denoise_layer (La  (None, 1, 3)              0         
 nguageDenoiseLayer)                                             
                                                                 
 listener (Functional)       (None, 1, 3)              59        
                                                                 
Total params: 119 (476.00 Byte)
Trainable params: 119 (476.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_______________________

In [7]:
inst = model.train(
    generator = gen,
    generations = 1,
    num_solutions = 50,
    num_parents_mating = 10,
)

c:\Users\nicho\PyProjects\CGAEL\.conda\Lib\site-packages\pygad\pygad.py:909: UserWarning: The steady-state parent (sss) selection operator is used despite that no parents are kept in the next generation.
  warnings.warn("The steady-state parent (sss) selection operator is used despite that no parents are kept in the next generation.")


Completed Generation: 1
Generating new batch...
Identifying best...


In [8]:
swatches = [Swatch.BLACK, Swatch.RED, Swatch.GREEN, Swatch.YELLOW, Swatch.BLUE, Swatch.MAGENTA, Swatch.CYAN, Swatch.WHITE]
samples = [sample_swatch(x) for x in swatches]
print(samples)

[[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]]


In [9]:
KEY_SWATCH = "swatch"
KEY_TEXT = "text"
KEY_IN = "input"
KEY_OUT = "output"
KEY_ROUND = "output_rounded"

swatches = [Swatch.BLACK, Swatch.RED, Swatch.GREEN, Swatch.YELLOW, Swatch.BLUE, Swatch.MAGENTA, Swatch.CYAN, Swatch.WHITE]
samples = np.array([[sample_swatch(x)] for x in swatches])
lang, out = model.model(samples)
text = [ts.decode(x) for x in lang]

d = {x:[] for x in [KEY_SWATCH, KEY_TEXT, KEY_IN, KEY_OUT, KEY_ROUND]}
for s, t, i, o in zip(swatches, text, samples, out):
    d[KEY_SWATCH].append(s)
    d[KEY_TEXT].append(t)
    d[KEY_IN].append(i)
    o = o.numpy()
    d[KEY_OUT].append(o)
    o = np.round(o)
    d[KEY_ROUND].append(o)
df = pd.DataFrame(data=d)
display(df)

,swatch,text,input,output,output_rounded
0,Swatch.BLACK,AHT,"[[0.0, 0.0, 0.0]]","[[0.851941, 0.07961328, 0.33238405]]","[[1.0, 0.0, 0.0]]"
1,Swatch.RED,AHA,"[[0.0, 0.0, 1.0]]","[[0.7766262, 0.49132678, 0.74484783]]","[[1.0, 0.0, 1.0]]"
2,Swatch.GREEN,AHC,"[[0.0, 1.0, 0.0]]","[[0.71936345, 0.30606592, 0.7708183]]","[[1.0, 0.0, 1.0]]"
3,Swatch.YELLOW,,"[[0.0, 1.0, 1.0]]","[[0.8642007, 0.9275416, 0.8173841]]","[[1.0, 1.0, 1.0]]"
4,Swatch.BLUE,AHA,"[[1.0, 0.0, 0.0]]","[[0.7766262, 0.49132678, 0.74484783]]","[[1.0, 0.0, 1.0]]"
5,Swatch.MAGENTA,,"[[1.0, 0.0, 1.0]]","[[0.8642007, 0.9275416, 0.8173841]]","[[1.0, 1.0, 1.0]]"
6,Swatch.CYAN,,"[[1.0, 1.0, 0.0]]","[[0.8642007, 0.9275416, 0.8173841]]","[[1.0, 1.0, 1.0]]"
7,Swatch.WHITE,,"[[1.0, 1.0, 1.0]]","[[0.8642007, 0.9275416, 0.8173841]]","[[1.0, 1.0, 1.0]]"
